In [1]:
import numpy as np
import cv2

In [4]:
from ex2_utils import Tracker, get_patch, extract_histogram, backproject_histogram, create_epanechnik_kernel
import math

In [ ]:
def mean_shift(frame, kernel, position, patch, nbins, q, eps=0.05):

    # Round to odd number
    kernel_size_x = math.ceil(kernel.shape[0] / 2.) * 2 - 1
    kernel_size_y = math.ceil(kernel.shape[1] / 2.) * 2 - 1

    # Caculate the derivatives of size of kernel
    xi_X = np.array([list(range(-math.floor(kernel_size_x / 2), math.floor(kernel_size_x / 2) + 1)) for i in range(kernel_size_y)])
    xi_Y = xi_X.T

    positions = []
    iter_count = 0
    while True:

        # Cut patch of the new position
        patch, _ = get_patch(frame, position, (kernel_size_x, kernel_size_y))
        
        # Extract histogram p using defined kernel
        hist = extract_histogram(patch, nbins, weights=kernel)
        hist_sum = sum(hist)
        p = np.array([i / hist_sum for i in hist])

        # Calculate weights v
        v = math.sqrt(q / (p + eps))

        # Calculate wi using backprojection
        wi = backproject_histogram(patch, v, nbins)

        # Calculate the changes in both x and y directions
        xk_X = np.sum(xi_X * wi) / np.sum(wi)
        xk_Y = np.sum(xi_Y * wi) / np.sum(wi)

        # Update the position accordingly
        position = (position[0] + xk_X, position[1] + xk_Y)
        positions.append(tuple(map(int, np.floor(position))))
        print(position)

        # Check if the algorithm converged
        if abs(xk_X) < eps and abs(xk_Y) < eps:
            break

        # Sanity check if the algorihtm does not converge
        if iter_count >= 500:
            break

        iter_count += 1

    position = tuple(map(int, np.floor(position)))
    return position, iter_count, positions

In [ ]:
class NCCTracker(Tracker):

    def initialize(self, image, region):

        if len(region) == 8:
            x_ = np.array(region[::2])
            y_ = np.array(region[1::2])
            region = [np.min(x_), np.min(y_), np.max(x_) - np.min(x_) + 1, np.max(y_) - np.min(y_) + 1]

        self.window = max(region[2], region[3]) * self.parameters.enlarge_factor

        left = max(region[0], 0)
        top = max(region[1], 0)

        right = min(region[0] + region[2], image.shape[1] - 1)
        bottom = min(region[1] + region[3], image.shape[0] - 1)

        self.patch = image[int(top):int(bottom), int(left):int(right)]
        self.position = (region[0] + region[2] / 2, region[1] + region[3] / 2)
        self.size = (region[2], region[3])

        # Epanechnikov kernel
        self.kernel = create_epanechnik_kernel(region[2], region[3], self.sigma)

        # Initial value of q (Normalised extracted histogram)
        hist = extract_histogram(self.patch, self.nbins, weights=self.kernel)
        hist_sum = sum(hist)
        self.q = np.array([i / hist_sum for i in hist])

    def track(self, image):

        left = max(round(self.position[0] - float(self.window) / 2), 0)
        top = max(round(self.position[1] - float(self.window) / 2), 0)

        right = min(round(self.position[0] + float(self.window) / 2), image.shape[1] - 1)
        bottom = min(round(self.position[1] + float(self.window) / 2), image.shape[0] - 1)

        if right - left < self.template.shape[1] or bottom - top < self.template.shape[0]:
            return [self.position[0] + self.size[0] / 2, self.position[1] + self.size[1] / 2, self.size[0], self.size[1]]

        # Calculate new positions of the patch using mean shift algorithm
        new_position, iter_count, _ = mean_shift(image, self.kernel, self.position, self.path, self.nbis, self.q, self.eps)

        # Update values of patch and position
        self.position = new_position
        self.path = get_patch(image, new_position, self.size)

        # Update q
        hist = extract_histogram(self.patch, self.nbins, weights=self.kernel)
        hist_sum = sum(hist)
        self.q = (1-self.alpha) * self.q + self.alpha * np.array([i / hist_sum for i in hist])

        # Return the values
        return [new_position[0], new_position[1], self.size[0], self.size[1]]

class NCCParams():
    def __init__(self):
        self.enlarge_factor = 2
        self.nbins = 16
        self.eps = 0.01
        self.sigma = 0.5
        self.alpha = 0.5

